# Haciendo peticiones http más "complejas"

Hasta ahora las peticiones que hemos realizado para hacer scraping son simples llamadas _GET_ a URLs concretas. Sin embargo en algunas ocasiones (por ejemplo cuando nos encontremos páginas en las que tenemos que interaccionar con formularios) o cuando queramos interactuar con una _API_ es posible que necesitemos hacer peticiones con otro verbo _http_ diferente o incluso mandar parámetros.

La siguiente página:

https://www.mscbs.gob.es/buscador/iniciar.do

nos permite hacer búsquedas en la web del Ministerio de Sanidad, Consumo y Bienestar Social. Si quisieramos scrapear la página de búsquedas podemos inspeccionar el código _HTML_ de la misma y localizar el formulario:

```
<form id="formBuscador" method="post" action="iniciar.do">
...
    <input type="text" name="search" value="" id="searchBsq" style="width: 20em;" class="campo_texto" aria-label="Término o términos de la consulta">
    <input type="submit" value="Buscar" class="boton">
    <input type="radio" name="buscar" value="msc" id="msc" checked="checked"><span class="negrita"><label for="msc">En el Ministerio de Sanidad, Consumo y Bienestar Social</label></span>				 								 							 						
    <input type="radio" name="buscar" value="internet" id="internet"><span class="negrita"><label for="internet">En Internet</label></span>
...
</form>
```
Vemos que la petición que se realizar es con el método ```POST``` y que hay varios campos que se pueden rellenar: ```search``` (el campo de búsqueda en sí) y ```buscar``` (que es un radio button con dos opciones: ```msc``` o ```internet```. Además la petición del formulario se va a hacer a la misma página: ```action="iniciar.do"```.


In [ ]:
# Ejemplo de búsqueda en la página descrita anteriormente

import requests

url = "https://www.mscbs.gob.es/buscador/iniciar.do"

parametros = {
    'search': 'COVID',  # Buscaremos el término 'COVID'
    'buscar': 'msc'     # Dentro de la página del ministerio
}

respuesta = requests.post(url, parametros)

html = respuesta.content.decode("utf-8")   

print(html)



# Beautiful Soup

Es otro parseador de páginas web muy utilizado por tener también una sintaxis sencilla y potente:

https://www.crummy.com/software/BeautifulSoup/bs4/doc/

Veamos un ejemplo sobre como podemos obtener de la página anterior los títulos de los documentos que ha devuelto la búsqueda que hemos hecho.


In [ ]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(html, 'html.parser')



In [ ]:
print(soup.prettify())  # Simplemente nos permite ver el documento mejor formatado (indentaciones, espacios en blanco...)


In [ ]:
# Buscamos un <div class="capaCentroBuscador">

cuadroResultados = soup.find('div', {"class": "capaCentroBuscador"})

# De ese cuadro extraemos todos los <li> (cada uno tiene un resultado de la búsqueda)

resultados = cuadroResultados.find_all('li')

for r in resultados:
    # Dentro de cada <li> tenemos un <p> con un <a href="URL_DE_LA_NOTICIA">
    urlNoticia = r.p.a['href']
    
    tituloNoticia = r.p.a.text
    
    print(tituloNoticia)
    print(urlNoticia)

# Trabajando con otros formatos: CSV

Sacar información de un documento CSV en principio debe ser mucho más fácil ya quees un documento que normalmente separa los datos de una tabla usando el salto de línea ```\n``` como delimitador de fila y la coma ortográfica ```,``` como separador de columnas. Sin embargo el uso de ```"``` dentro de los campos puede dificultar la división de cada línea en distintos campos. Por suerte ya tenemos funciones preparadas que son capaces de entender este formato y convertirlo a un formato más amigable para Python.

In [ ]:
import requests
from io import StringIO
import csv
import matplotlib.pyplot as plt

url="https://covid19.isciii.es/resources/serie_historica_acumulados.csv"

respuesta = requests.get(url)

contenidoCSV = respuesta.content.decode("iso-8859-1")   # Lo podemos codificar a una cadena "normal"

#print(contenidoCSV)

results = []
f = StringIO(contenidoCSV)
reader = csv.reader(f, delimiter=',')

fila = 0

for row in reader: # each row is a list
    if fila != 0:  # Ignoramos la primera fila porque es el nombre de las columnas
        results.append(row)
        
    fila += 1
        
results = results[:-8]  # Eliminamos las 8 ultimas filas porque es un texto que no tiene que ver con los datos


#print(results)

# Ahora que tenemos los datos vamos a hacer unas gráficas

# Recorremos los resultados creando estructuras de datos más comodas para los gráficos

ccaa = []
fechas = []
casosTotales = {}
pcr = {}
testAC = {}
hospitalizados = {}
uci = {}
fallecidos = {}
recuperados = {}


for r in results:
    comunidad = r[0]
    fecha = r[1]
    t = 0 if not r[2] else int(r[2])
    p = 0 if not r[3] else int(r[3])
    a = 0 if not r[4] else int(r[4])
    h = 0 if not r[5] else int(r[5])
    u = 0 if not r[6] else int(r[6])
    f = 0 if not r[7] else int(r[7])
    r = 0 if not r[8] else int(r[8])

    if not comunidad in ccaa:
        ccaa.append(comunidad)
        
        casosTotales[comunidad] = []
        pcr[comunidad] = []
        testAC[comunidad] = []
        hospitalizados[comunidad] = []
        uci[comunidad] = []
        fallecidos[comunidad] = []
        recuperados[comunidad] = []

    if not fecha in fechas:
        fechas.append(fecha)
        
    casosTotales[comunidad].append(t)
    pcr[comunidad].append(p)
    testAC[comunidad].append(a)
    hospitalizados[comunidad].append(h)
    uci[comunidad].append(u)
    fallecidos[comunidad].append(f)
    recuperados[comunidad].append(r)
    
    
variableGrafica = fallecidos
variableTexto = "Fallecidos"
comunidadResaltada = "AN"

#print(ccaa)
#print(fechas)
#print(fallecidos['AN'])

plt.style.use('seaborn-darkgrid')
my_dpi=96
plt.figure(figsize=(1000/my_dpi, 800/my_dpi), dpi=my_dpi)

for comunidad in ccaa:
   plt.plot(fechas, variableGrafica[comunidad], marker='', color='grey', linewidth=1, alpha=0.4)
 
plt.plot(fechas, variableGrafica[comunidadResaltada], marker='', color='orange', linewidth=4, alpha=0.7)


plt.title(variableTexto + " de " + comunidadResaltada + " frente a las demás CC.AA.", loc='left', fontsize=12, fontweight=0, color='orange')
plt.xlabel("Fecha")
plt.ylabel("")
plt.xticks(range(0, len(fechas), 1), fechas, rotation=90)


plt.show()

# Trabajando con otros formatos: _JSON_

Este formato se ha popularizado sobretodo para _APIs_ de Internet. Por ejemplo, el servicio gratuito _Nominatim_ permite hacer búsquedas de latitudes y longitudes a partir de consultas de texto habituales. Por ejemplo, podemos encontrar la latitud y la longitud de un edificio o de una calle:

https://nominatim.org/release-docs/develop/

Tal y como se nos dice en la documentación, para hacer una consulta tenemos varios parámetros que le podemos pasar a la petición de la página:

https://nominatim.openstreetmaps.org/search

Por ahora los parámetros que nos interesan son ```q``` donde pondremos la descripción textual de la calle o edificio que buscamos y ```format``` con el valor ```json``` para que nos de la información en este formato.

In [ ]:
import requests
import json 

url = "https://nominatim.openstreetmaps.org/search"

parametros = {
    'q': 'Facultad de Ciencias',  
    'format': 'json'   
}

respuesta = requests.get(url, parametros)

textoJson = respuesta.content.decode("utf-8")   

#print(textoJson)

resultados = json.loads(textoJson)

for r in resultados:
    print("+ Nombre: " + r['display_name'])
    print("  + Coordenadas: " + "(" + r['lat'] + ", " + r['lon'] + ")")
    print("  + Tipo: " + r['type'])
    print("  + Clase: " + r['class'])
    